In [50]:
#importing necessary libraries

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
#importing all libraries needed
import seaborn as sns
import numpy as np
import os
from sklearn.metrics import confusion_matrix
import seaborn as sn; sn.set(font_scale=1.4)
from sklearn.utils import shuffle           
import matplotlib.pyplot as plt             
import cv2                                 
import tensorflow as tf                
from tqdm import tqdm
import io
import os
#import splitfolders
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report , confusion_matrix
%matplotlib inline

#model
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.layers import Conv2D,MaxPool2D,Dense,Dropout,Flatten,GlobalAveragePooling2D,BatchNormalization
from tensorflow.keras.activations import relu,softmax
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Input, Model , models
from tensorflow.keras.applications import EfficientNetB3,InceptionV3, ResNet50, DenseNet121,DenseNet201,EfficientNetB6, VGG16, VGG19, ResNet101V2
from tensorflow.keras.losses import MeanAbsoluteError, MeanAbsolutePercentageError
from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau
from tensorflow.keras.utils import plot_model
from tensorflow.keras.preprocessing import image

We have a csv file with details like product price, product brand, product image. We will load this dataset now.

In [38]:
# df = pd.read_csv(io.BytesIO(uploaded['product_details.csv']),index_col=0)
#df = pd.read_csv('product_details.csv',index_col=0)

In [39]:
#Defining the class names in our classification model. It will be used later in this source code
class_names=['balenciaga','chanel','christian dior', 'fendi', 'gucci', 'louis vuitton','prada','saint laurent']

#Assigning index to class names for visualization later 
class_names_label = {class_name:i for i, class_name in enumerate(class_names)}

# number of classes
nb_classes = len(class_names)

# converting all our images to (150,150) dimension
IMAGE_SIZE = (150,150)

In [40]:
#df.head()

In [41]:
#This code was written as some file names got edited in the kaggle notebook which we used, hence we used this code to convert the file names that are compatible with kaggle
#y = df['image_names(\'jpg\')_mod'].to_list()
#new_x = []
#for i in y:
#    w = i.replace('\'','_')
#    new_x.append(w)
#df['image_names(\'jpg\')'] = new_x

In [42]:
#Our dataframe
#df.head()

In [43]:
#function to load images from the directories into numpy arrays. We also store the image labels in numpy arrays. 
#It can be used for visualizations for images.
'''def load_data():
    
    datasets = ["train","val","test"]
    output = []
    
# Iterate through dataset
    for dataset in datasets:
        
        images = []
        labels = []
        prices = []
        
        print("Loading {}".format(dataset))
        
      # Iterate through each folder corresponding to a category
        for folder in os.listdir(dataset):
            label = class_names_label[folder]
            
          # Iterate through each image in our folder
            for file in tqdm(os.listdir(os.path.join(dataset, folder))):
                
              # Get the path name of the image
                img_path = os.path.join(os.path.join(dataset, folder), file)
                
              # Open and resize the img
                image = cv2.imread(img_path)
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                image = cv2.resize(image, IMAGE_SIZE) 
                
                row = df[df["image_names('jpg')"] == file]
                #print(row["image_names('jpg')"])
                if row.empty:
                    price = 0
                else:
                    price = int(row['product_price'].item().strip('$').replace(',',''))
                    prices.append(price)
                    images.append(image)
                    labels.append(label)
                #print('done')
              # Append the image and its corresponding label to the output
                #images.append(image)
                #labels.append(label)
                #prices.append(price)
                
        images = np.array(images, dtype = 'float32')
        labels = np.array(labels, dtype = 'int32')
        prices = np.array(prices, dtype = 'float32')   
        
        output.append((images, labels,prices))
    
    return output
'''

'def load_data():\n    \n    datasets = ["train","val","test"]\n    output = []\n    \n# Iterate through dataset\n    for dataset in datasets:\n        \n        images = []\n        labels = []\n        prices = []\n        \n        print("Loading {}".format(dataset))\n        \n      # Iterate through each folder corresponding to a category\n        for folder in os.listdir(dataset):\n            label = class_names_label[folder]\n            \n          # Iterate through each image in our folder\n            for file in tqdm(os.listdir(os.path.join(dataset, folder))):\n                \n              # Get the path name of the image\n                img_path = os.path.join(os.path.join(dataset, folder), file)\n                \n              # Open and resize the img\n                image = cv2.imread(img_path)\n                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)\n                image = cv2.resize(image, IMAGE_SIZE) \n                \n                row = df[df["imag

In [44]:
#loading the data (optional)
#(train_images,train_labels,train_prices),(val_images,val_labels,val_prices) (test_images,test_labels,test_prices) = load_data()

In [45]:
#print(len(train_images))
#print(len(train_labels))
#print(len(train_prices))

In [46]:
'''fig = plt.figure(figsize=(20,20))
for i in range(9):
    index = np.random.randint(train_images.shape[0])
    plt.subplot(3,3,i+1)
    plt.imshow(train_images[index])
    plt.title('Image #{} : '.format(index)+class_names[train_labels[index]] +", Price: "+ str(train_price[index]))
    plt.axis('off')
'''

'fig = plt.figure(figsize=(20,20))\nfor i in range(9):\n    index = np.random.randint(train_images.shape[0])\n    plt.subplot(3,3,i+1)\n    plt.imshow(train_images[index])\n    plt.title(\'Image #{} : \'.format(index)+class_names[train_labels[index]] +", Price: "+ str(train_price[index]))\n    plt.axis(\'off\')\n'

Image Augmentation

In [47]:
#Creating our image data generator
datagen = ImageDataGenerator(
        rescale=1/255.0,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

#training our model on augmented images
batch_size = 128
train_generator = datagen.flow_from_directory('C:\\Users\\hdeok\\Downloads\\Data\\train',
                                             target_size=IMAGE_SIZE,
                                             batch_size=batch_size,
                                             shuffle=True,
                                             class_mode='categorical')

validation_generator = datagen.flow_from_directory('C:\\Users\\hdeok\\Downloads\\Data\\val',
                                             target_size=IMAGE_SIZE,
                                             batch_size=batch_size,
                                             shuffle=True,
                                             class_mode='categorical')
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        'C:\\Users\\hdeok\\Downloads\\Data\\test',
        target_size=(150,150),
        batch_size=1024,
        class_mode='categorical')

Found 13201 images belonging to 8 classes.
Found 929 images belonging to 8 classes.
Found 949 images belonging to 8 classes.


Training our custom model and checking it against our dataset.

In [33]:
#Creating our baseline model.
model = Sequential([Conv2D(32, (3,3),activation='relu',input_shape=(180,180,3)),
                   BatchNormalization(),
                   MaxPool2D(2,2),
                   Conv2D(64, (3,3) ,activation='relu'),
                   BatchNormalization(),
                   MaxPool2D(2,2),
                   Conv2D(128, (3,3) ,activation='relu'),
                   BatchNormalization(),
                   MaxPool2D(2,2),
                   Conv2D(256, (3,3) ,activation='relu'),
                   BatchNormalization(),
                   MaxPool2D(2,2),
                   Conv2D(512, (3,3) ,activation='relu'),
                   BatchNormalization(),
                   GlobalAveragePooling2D(), 
                   Flatten(),
                   Dense(512, activation='relu'),
                   Dropout(0.4),
                   Dense(256, activation='relu'),
                   #Dense(128,activation='relu'),
                   #Dense(64,activation='relu'),
                   Dense(8,activation='softmax')])

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['acc'])

In [34]:
class MyCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('acc') > 0.99:
            print("\nReached accuracy threshold! Terminating training.")
            self.model.stop_training = True      
my_callback = MyCallback()
rop_callback = ReduceLROnPlateau(monitor="val_acc", patience=3)
es_callback = tf.keras.callbacks.EarlyStopping(
                monitor='val_acc',
                #min_delta=1,
                patience=10,
                restore_best_weights=True)
checkpoint_filepath = './'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

METRICS = [tf.keras.metrics.CategoricalAccuracy(name='acc'),
           tf.keras.metrics.AUC(name='auc')]

CALLBACKS = [my_callback, rop_callback, es_callback,model_checkpoint_callback]

In [35]:
#model summary
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_15 (Conv2D)          (None, 178, 178, 32)      896       
                                                                 
 batch_normalization_15 (Bat  (None, 178, 178, 32)     128       
 chNormalization)                                                
                                                                 
 max_pooling2d_15 (MaxPoolin  (None, 89, 89, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_16 (Conv2D)          (None, 87, 87, 64)        18496     
                                                                 
 batch_normalization_16 (Bat  (None, 87, 87, 64)       256       
 chNormalization)                                                
                                                      

In [36]:
#training the model
history = model.fit(train_generator,steps_per_epoch=13201 // batch_size,
                    validation_data=validation_generator, 
                    validation_steps=929 // batch_size,
callbacks=CALLBACKS, epochs=100, batch_size=128)

Epoch 1/100
  6/103 [>.............................] - ETA: 12:17 - loss: 2.1420 - acc: 0.1875

KeyboardInterrupt: 

In [ ]:
# accuracy plot

plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epochs')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
# loss plot
plt.subplot()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epochs')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
#evaluating model performance against test data
model.evaluate(test_generator)

In [ ]:
#Saving the model
model.save('Custom_model.h5')

We will now plot the confusion matrix for our predicted values

In [ ]:
test_data, test_labels = test_generator.next()

In [ ]:
pred_lab = model.predict(test_data)

# Classification report block
def roundoff(arr):
    arr[np.argwhere(arr != arr.max())] = 0
    arr[np.argwhere(arr == arr.max())] = 1
    return arr

for labels in pred_lab:
    labels = roundoff(labels)

print(classification_report(test_labels, pred_lab, target_names=class_names))

# Confussion Matrix block
pred_ls = np.argmax(pred_lab, axis=1)
test_ls = np.argmax(test_labels, axis=1)

conf_arr = confusion_matrix(test_ls, pred_ls)

plt.figure(figsize=(8, 6), dpi=80, facecolor='w', edgecolor='k')

ax = sns.heatmap(conf_arr, cmap='PuBuGn', annot=True, fmt='d', xticklabels= class_names,
                yticklabels=class_names)

plt.title('Bag Classification')
plt.xlabel('Prediction')
plt.ylabel('Truth')
plt.show(ax)

InceptionV3 model

In [48]:
Inception = InceptionV3(weights='imagenet',input_shape=(150, 150, 3), include_top=False)

for layer in Inception.layers:
    layer.trainable=False
    
InceptionV3_model=Sequential()
InceptionV3_model.add(Inception)
InceptionV3_model.add(Flatten())
InceptionV3_model.add(Dense(1024, activation='relu'))
InceptionV3_model.add(Dense(512, activation='relu'))
#DenseNet121_model.add(Dense(256, activation='relu'))
InceptionV3_model.add(Dense(8,activation='softmax'))

#callbacks for training
my_callback = MyCallback()
rop_callback = ReduceLROnPlateau(monitor="val_loss", patience=3)
es_callback = tf.keras.callbacks.EarlyStopping(
                monitor='val_loss',
                patience=15,
                restore_best_weights=True)

METRICS = [tf.keras.metrics.CategoricalAccuracy(name='acc'),
           tf.keras.metrics.AUC(name='auc')]

CALLBACKS = [my_callback, rop_callback, es_callback]
    
InceptionV3_model.compile(optimizer='adam',
                              loss='categorical_crossentropy',
                              metrics=METRICS)

InceptionV3_model.summary()

87924736/87910968 [==============================] - 20s 0us/step
Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 3, 3, 2048)        21802784  
                                                                 
 flatten_4 (Flatten)         (None, 18432)             0         
                                                                 
 dense_12 (Dense)            (None, 1024)              18875392  
                                                                 
 dense_13 (Dense)            (None, 512)               524800    
                                                                 
 dense_14 (Dense)            (None, 8)                 4104      
                                                                 
Total params: 41,207,080
Trainable params: 19,404,296
Non-trainable params: 21,802,784
_________________________________

In [49]:
#training our model
history = InceptionV3_model.fit(train_generator,
    steps_per_epoch=13201  // batch_size,  # number of train images/ batch size
    validation_data=validation_generator,
    validation_steps=929  // batch_size, #number of validation images/ batch size
    epochs=100)
#history = model.fit(train_images, train_labels, epochs=100, batch_size=128,validation_split=0.2)

Epoch 1/100
 12/103 [==>...........................] - ETA: 6:16 - loss: 33.5349 - acc: 0.1445 - auc: 0.5135

KeyboardInterrupt: 

In [ ]:
# accuracy plot

plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epochs')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
# loss plot
plt.subplot()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epochs')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
#evaluating model performance against test data
InceptionV3_model.evaluate(test_generator)

In [ ]:
#getting test data and labels for classification report
test_data, test_labels = test_generator.next()

In [ ]:
pred_lab = InceptionV3_model.predict(test_data)

# Classification report block
for labels in pred_lab:
    labels = roundoff(labels)

print(classification_report(test_labels, pred_lab, target_names=class_names))

# Confussion Matrix block
pred_ls = np.argmax(pred_lab, axis=1)
test_ls = np.argmax(test_labels, axis=1)

conf_arr = confusion_matrix(test_ls, pred_ls)

plt.figure(figsize=(8, 6), dpi=80, facecolor='w', edgecolor='k')

ax = sns.heatmap(conf_arr, cmap='PuBuGn', annot=True, fmt='d', xticklabels= class_names,
                yticklabels=class_names)

plt.title('Bag Classification')
plt.xlabel('Prediction')
plt.ylabel('Truth')
plt.show(ax)

DenseNet201 model

In [51]:
DNET = DenseNet201(input_shape=(150, 150, 3), include_top=False, weights="imagenet")

for layer in DNET.layers:
    layer.trainable=False
    
DenseNet201_model=Sequential()
DenseNet201_model.add(DNET)
DenseNet201_model.add(Flatten())
DenseNet201_model.add(Dense(1024, activation='relu'))
DenseNet201_model.add(Dense(512, activation='relu'))
#DenseNet121_model.add(Dense(256, activation='relu'))
DenseNet201_model.add(Dense(8,activation='softmax'))

#callbacks for training
my_callback = MyCallback()
rop_callback = ReduceLROnPlateau(monitor="val_loss", patience=3)
es_callback = tf.keras.callbacks.EarlyStopping(
                monitor='val_loss',
                patience=10,
                restore_best_weights=True)

METRICS = [tf.keras.metrics.CategoricalAccuracy(name='acc'),
           tf.keras.metrics.AUC(name='auc')]

CALLBACKS = [my_callback, rop_callback, es_callback]
    
DenseNet201_model.compile(optimizer='adam',
                              loss='categorical_crossentropy',
                              metrics=METRICS)

DenseNet201_model.summary()

74850304/74836368 [==============================] - 17s 0us/step
Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 densenet201 (Functional)    (None, 4, 4, 1920)        18321984  
                                                                 
 flatten_5 (Flatten)         (None, 30720)             0         
                                                                 
 dense_15 (Dense)            (None, 1024)              31458304  
                                                                 
 dense_16 (Dense)            (None, 512)               524800    
                                                                 
 dense_17 (Dense)            (None, 8)                 4104      
                                                                 
Total params: 50,309,192
Trainable params: 31,987,208
Non-trainable params: 18,321,984
_________________________________

In [52]:
#training our model
history = DenseNet201_model.fit(train_generator,
    steps_per_epoch=13201  // batch_size,  # number of train images/ batch size
    validation_data=validation_generator,
    validation_steps=929  // batch_size, #number of validation images/ batch size
    epochs=100)
#history = model.fit(train_images, train_labels, epochs=100, batch_size=128,validation_split=0.2)

Epoch 1/100
 12/103 [==>...........................] - ETA: 17:39 - loss: 29.5074 - acc: 0.1693 - auc: 0.5310

KeyboardInterrupt: 

In [ ]:
# accuracy plot

plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epochs')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
# loss plot
plt.subplot()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epochs')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
#evaluating model performance against test data
DenseNet201_model.evaluate(test_generator)

In [ ]:
#Saving the model
DenseNet201_model.save("DenseNet_201.h5")

In [ ]:
test_data, test_labels = test_generator.next()

In [ ]:
pred_lab = DenseNet201_model.predict(test_data)

# Classification report block
for labels in pred_lab:
    labels = roundoff(labels)

print(classification_report(test_labels, pred_lab, target_names=class_names))

# Confussion Matrix block
pred_ls = np.argmax(pred_lab, axis=1)
test_ls = np.argmax(test_labels, axis=1)

conf_arr = confusion_matrix(test_ls, pred_ls)

plt.figure(figsize=(8, 6), dpi=80, facecolor='w', edgecolor='k')

ax = sns.heatmap(conf_arr, cmap='PuBuGn', annot=True, fmt='d', xticklabels= class_names,
                yticklabels=class_names)

plt.title('Bag Classification')
plt.xlabel('Prediction')
plt.ylabel('Truth')
plt.show(ax)